### Notebook to demonstrate Auto-Labeling workflow

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)


### The workflow in a nutshell

- Creating a dataset
- Upload dataset to the service
- Getting a PTM from NGC
- Model Actions
    - Train (Normal/AutoML)
    - Evaluate
    - Inference on TAO

### Table of contents

1. [Create datasets ](#head-1)
1. [List the created datasets](#head-2)
1. [Create model ](#head-4)
1. [List models](#head-5)
1. [Assign train, eval datasets](#head-6)
1. [Assign PTM](#head-7)
1. [View hyperparameters that are enabled by default](#head-8)
1. [Set AutoML related configurations](#head-9)
1. [Actions](#head-10)
1. [Train](#head-11)
1. [Evaluate](#head-12)
1. [TAO inference](#head-20)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import requests
import uuid
import time
from IPython.display import clear_output

### FIXME

1. Assign a workdir in FIXME 1
1. Assign the ip_address and port_number in FIXME 2 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
1. Assign the ngc_api_key variable in FIXME 3
1. (Optional) Enable AutoML if needed in FIXME 4
1. Choose between default and custom dataset in FIXME 5
1. Assign path of DATA_DIR in FIXME 6
1. Choose between Bayesian and Hyperband automl_algorithm in FIXME 7 (If automl was enabled in FIXME4)

In [ ]:
model_name = "mal"
workdir = "workdir_auto_labeling" # FIXME1
host_url = "http://<ip_address>:<port_number>" # FIXME2 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>" # FIXME3 example: (Add NGC API key)

In [ ]:
automl_enabled = False # FIXME4 set to TRUE if you want to run automl for the model chosen in the previous cell

In [ ]:
# Exchange NGC_API_KEY for JWT
response = requests.get(f"{host_url}/api/v1/login/{ngc_api_key}")
user_id = response.json()["user_id"]
print("User ID",user_id)
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Create datasets <a class="anchor" id="head-1"></a>

We will be using the `COCO dataset`. `download_coco.sh` script from dataset prepare will be used to download and unzip the coco2017 dataset from [here](https://cocodataset.org/#download)


**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── annotations.json
├── images
    ├── image_name_1.jpg
    ├── image_name_2.jpg
    ├── ...

```

In [ ]:
dataset_to_be_used = "default" #FIXME5 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = model_name # FIXME6
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

### Download dataset

In [ ]:
if dataset_to_be_used == "default":
    !bash dataset_prepare/coco/download_coco.sh $DATA_DIR
    # Remove existing data
    !rm -rf $DATA_DIR/train2017/images
    !rm -rf $DATA_DIR/val2017/images
    # Rearrange data in the required format
    !mkdir -p $DATA_DIR/train2017/
    !mkdir -p $DATA_DIR/val2017/
    !mv $DATA_DIR/raw-data/train2017 $DATA_DIR/train2017/images
    !mv $DATA_DIR/raw-data/annotations/instances_train2017.json $DATA_DIR/train2017/annotations.json
    !mv $DATA_DIR/raw-data/val2017 $DATA_DIR/val2017/images
    !mv $DATA_DIR/raw-data/annotations/instances_val2017.json $DATA_DIR/val2017/annotations.json


### Verify the downloaded dataset

In [ ]:
!if [ ! -d $DATA_DIR/train2017/images ]; then echo 'Images folder not found'; else echo 'Found images folder';fi
!if [ ! -f $DATA_DIR/train2017/annotations.json ]; then echo 'annotations file not found'; else echo 'Found annotations file';fi
!if [ ! -d $DATA_DIR/val2017/images ]; then echo 'Images folder not found'; else echo 'Found images folder';fi
!if [ ! -f $DATA_DIR/val2017/annotations.json ]; then echo 'annotations file not found'; else echo 'Found annotations file';fi

In [ ]:
!tar -C $DATA_DIR/train2017 -czf $DATA_DIR/coco_train.tar.gz images annotations.json
!tar -C $DATA_DIR/val2017 -czf $DATA_DIR/coco_val.tar.gz images annotations.json

In [ ]:
train_dataset_path = f"{DATA_DIR}/coco_train.tar.gz"
eval_dataset_path= f"{DATA_DIR}/coco_val.tar.gz"

In [ ]:
# Create train dataset
ds_type = "instance_segmentation"
ds_format = "coco"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())

dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Train dataset",
                       "description":"My train dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(train_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create eval dataset
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())

eval_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Evaluation dataset",
                       "description":"My eval dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(eval_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{eval_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### List the created datasets <a class="anchor" id="head-2"></a>

In [ ]:
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Create model <a class="anchor" id="head-4"></a>

In [ ]:
encode_key = "tlt_encode"
data = json.dumps({"network_arch":model_name,"encryption_key":encode_key})

endpoint = f"{base_url}/model"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
model_id = response.json()["id"]

### List models <a class="anchor" id="head-5"></a>

In [ ]:
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign train, eval datasets <a class="anchor" id="head-6"></a>

- Note: make sure the order for train_datasets is [source ID, target ID]
- eval_dataset is kept same as target for demo purposes

In [ ]:
dataset_information = {"train_datasets":[dataset_id],
                       "eval_dataset":eval_dataset_id,
                       "inference_dataset":eval_dataset_id}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign PTM <a class="anchor" id="head-7"></a>

Search for the PTM on NGC for the Segmentation model chosen

In [ ]:
# List all pretrained models for the chosen network architecture
model_list = f"{base_url}/model"
response = requests.get(model_list, headers=headers)

response_json = response.json()

for rsp in response_json:
    if rsp["network_arch"] == model_name:
        if "encryption_key" not in rsp.keys():
            print(f'PTM Name: {rsp["name"]}; PTM version: {rsp["version"]}; NGC PATH: {rsp["ngc_path"]}; Additional info: {rsp["additional_id_info"]}')

In [ ]:
# Assigning pretrained models to different networks
# From the output of previous cell make the appropriate changes to this map if you want to change the default PTM backbone.
# Changing the default backbone here requires changing default spec/config during train/eval etc like for example
# If you are changing the ptm to resnet34, then you have to modify the config key num_layers if it exists to 34 manually
pretrained_map = {"mal" : "mask_auto_label:trainable_v1.0"}
no_ptm_models = set([])

In [ ]:
# Get pretrained model
if model_name not in no_ptm_models:
    model_list = f"{base_url}/model"
    response = requests.get(model_list, headers=headers)

    response_json = response.json()

    # Search for ptm with given ngc path
    ptm = []
    for rsp in response_json:
        if rsp["network_arch"] == model_name and rsp["ngc_path"].endswith(pretrained_map[model_name]):
            ptm_id = rsp["id"]
            ptm = [ptm_id]
            print("Metadata for model with requested NGC Path")
            print(rsp)
            break

In [ ]:
if model_name not in no_ptm_models:
    ptm_information = {"ptm":ptm}
    data = json.dumps(ptm_information)

    endpoint = f"{base_url}/model/{model_id}"

    response = requests.patch(endpoint, data=data, headers=headers)

    print(response)
    print(response.json())

### View hyperparameters that are enabled for AutoML by default <a class="anchor" id="head-8"></a>

In [ ]:
if automl_enabled:
    # Get default spec schema
    endpoint = f"{base_url}/model/{model_id}/specs/train/schema"
    response = requests.get(endpoint, headers=headers)
    specs = response.json()["automl_default_parameters"]
    print(json.dumps(specs, sort_keys=True, indent=4))

### Set AutoML related configurations <a class="anchor" id="head-9"></a>
Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters: [Mask RCNN](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_action_specs.html#id39), [Unet](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_action_specs.html#id69), [Segformer](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_action_specs.html#id69)

In [ ]:
if automl_enabled:
    # Choose automl algorithm between "Bayesian" and "HyperBand".
    automl_algorithm="Bayesian" # FIXME8 example: Bayesian/HyperBand

    metric="kpi" #don't change, more metrics will be supported in the future

    additional_automl_parameters = [] #Refer to parameter list mentioned in the above links and add any extra parameter in addition to the default enabled ones
    remove_default_automl_parameters = [] #Remove any hyperparameters that are enabled by default for AutoML

    automl_information = {"automl_enabled":automl_enabled,
                          "automl_algorithm":automl_algorithm,
                          "metric":metric,
                          "automl_add_hyperparameters":str(additional_automl_parameters),
                          "automl_remove_hyperparameters":str(remove_default_automl_parameters)
                         }
    data = json.dumps(automl_information)

    endpoint = f"{base_url}/model/{model_id}"

    response = requests.patch(endpoint, data=data, headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

### Actions <a class="anchor" id="head-10"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download endpoint (if needed)

In [ ]:
job_map = {}

### Train <a class="anchor" id="head-11"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Override any of the parameters listed in the previous cell as required
specs["gpu_ids"] = [0]

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/train"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["train"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["train"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
# For automl: Training times for different models benchmarked on 1 GPU V100 machine can be found here: https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#results-of-automl-experiments

job_id = job_map['train']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:    
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor job status by repeatedly running this cell' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# if automl_enabled:
#     job_id = job_map['train']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

#     response = requests.post(endpoint, headers=headers)

#     print(response)
#     print(response.json())

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor job status by repeatedly running this cell' cell above (4th cell above from this cell)
# if automl_enabled:
#     job_id = job_map['train']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/resume"

#     response = requests.post(endpoint, headers=headers)

#     print(response)
#     print(response.json())

In [ ]:
# Download job contents once the above job shows "Done" status
# Download output of train job (Note: will take time)
job_id = job_map["train"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# View the checkpoints generated for the training job and for automl jobs, in addition view: best performing model's config and the results of all automl experiments

if automl_enabled:
    !python3 -m pip install pandas==1.5.1
    import pandas as pd
    model_downloaded_path = f"{model_downloaded_path}/best_model"

if os.path.exists(model_downloaded_path):        
    #List the binary model file
    print("\nCheckpoints for the training experiment")
    if os.path.exists(model_downloaded_path+"/weights") and len(os.listdir(model_downloaded_path+"/weights")) > 0:
        print(f"Folder: {model_downloaded_path}/weights")
        print("Files:", os.listdir(model_downloaded_path+"/weights"))
    else:
        print(f"Folder: {model_downloaded_path}")
        print("Files:", os.listdir(model_downloaded_path))

    if automl_enabled:
        experiment_artifacts = json.load(open(f"{model_downloaded_path}/controller.json","r"))
        data_frame = pd.DataFrame(experiment_artifacts)
        # Print experiment id/number and the corresponding result
        print("\nResults of all experiments")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
            print(data_frame[["id","result"]])

        print("\nConfig/Spec file for the best performing experiment (recommendation_id.kitti with the maximum result value in the dataframe)")
        # List the recommendation config file of the best performing checkpoint(recommendation_id.kitti with the maximum result value in the dataframe)
        !ls {model_downloaded_path}/*.kitti 

### Evaluate <a class="anchor" id="head-12"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/evaluate/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs if required

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/evaluate"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["evaluate"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["evaluate"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['evaluate']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:    
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### TAO inference <a class="anchor" id="head-20"></a>

- Run inference on a set of images using the .tlt model created at train step

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/inference/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to specs if necessary

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/inference"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["inference_tlt"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['inference_tlt']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:    
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["inference_tlt"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
inference_out_path = f"{workdir}/{job_id}"

In [ ]:
# Inference output must be here
!ls {inference_out_path}/inference.json